# Introduciton to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natual language (could be sequences test or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [ ]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## Get helper funcitons

In [ ]:
!wget https://raw.githubusercontent.com/arrshsh/ML-and-DS/main/TensorFlow/helper_functions.py

--2024-12-23 02:26:16--  https://raw.githubusercontent.com/arrshsh/ML-and-DS/main/TensorFlow/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.2’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-12-23 02:26:16 (48.5 MB/s) - ‘helper_functions.py.2’ saved [10246/10246]



In [ ]:
# Import the required functions
from helper_functions import unzip_data, plot_loss_curves, compare_historys

## Get a text dataset

the dataset we're going to use is Kaggle's introduciton to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

See the original source here: https://www.kaggle.com/competitions/nlp-getting-started

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the data
unzip_data("nlp_getting_started.zip")

--2024-12-23 02:26:16--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.101.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.2’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-12-23 02:26:16 (110 MB/s) - ‘nlp_getting_started.zip.2’ saved [607343/607343]



## Visualising text data

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# What does test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many samples of each class do we have?
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [ ]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)- 5) # Create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index: random_index + 5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "{real disaster}" if target > 0 else "{not real disaster}")
  print(f"Text: \n {text}\n")
  print("-------------------------------------------\n")

Target: 0 {not real disaster}
Text: 
 #ika #tuning Soup #diet recipes | fat burning soup recipes: http://t.co/8r5vpAoo5z Fat Burning Soup Diet Recip http://t.co/JvcxB75DrJ

-------------------------------------------

Target: 1 {real disaster}
Text: 
 A Look at State Actions a Year After #Ferguson's Upheaval http://t.co/qwSbVfLPE1

-------------------------------------------

Target: 1 {real disaster}
Text: 
 Looks like a war zone outside. What's going on?

-------------------------------------------

Target: 1 {real disaster}
Text: 
 @WKRN 
Parents of Colorado theater shooting victim fear copycat massacre

http://t.co/LvlH3W3aWO
#Antioch

http://t.co/vIwXY1XDYK

-------------------------------------------

Target: 0 {not real disaster}
Text: 
 I sent my emails why are the TRINNA hold me hostage  rapping me up ??

-------------------------------------------



### Splitting data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split # this function accepts input as numpy
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
train_df_shuffled["target"].to_numpy(),
test_size = 0.1,
random_state = 42)

In [ ]:
# Check lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [ ]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Convert text into numbers

When dealing with a text problem, one of the things you'll have to do before you can build a model is to convert your text to numbers. There are a few ways to do this, namely:

* Tokenisation: direct mapping of token (a token could be a word or character) to number.
* Embedding: create a matrix of feature vector for each token (the size of feature vector can be defined and this embedding can be learned and updated by our models during the training phase)

### Text vectorisation (tokenisation)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [ ]:
# Use default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens = 10000, # How many words in the vocab? None means no cap on the limit, but can be used None only when pad_to_max_tokens is False
                                    standardize = "lower_and_strip_punctuation",
                                    split= "whitespace",
                                    ngrams = None, # Create groups of n-words
                                    output_mode = "int", # How to map tokens to numbers
                                    output_sequence_length = None, # How long do you want the sequences to be?
                                    pad_to_max_tokens = True)

In [ ]:
train_sentences[0].split(), len(train_sentences[0].split())

(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [ ]:
# Find the average number of tokens (words) in the training tweets
round(sum(len(i.split()) for i in train_sentences) / len(train_sentences))

15

In [ ]:
# Setup text vectorization variables
max_vocab_length = 10000 # Pick up the 10000 most common tokens from the data
max_length = 15 # The number of words from a tweet that our model sees. For instance if a tweet has 30 words, model jujst sees the first 15 tokens
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [ ]:
# Fit the text vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a fllod in the town!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3,   1,   4,   2, 801,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose random sentence from the dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence} \n\n\
        Vectorized text:")
text_vectorizer([random_sentence])

Original text: 
 Japan on Thursday marks the 70th anniversary of the atomic bombing of Hiroshima with the most senior official from Washington ever scheduleÛ_ 

        Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 224,   11, 1056,  883,    2, 1046,  521,    6,    2,  210,  156,
           6,  130,   14,    2]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an embedding using an Embedding Layer

For the purpose, we'll be using TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding




Some important parameters of the Embedding layer are:






* `input_dim`: the size of the vocabulary
* `output_dim`: the size of the output embedding vectors, for instance a value of 100 would mean that each token is represented by a vector 100 long
* `input_length`: length of the sequences being passed to the embedding layer

In [ ]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             embeddings_initializer = "uniform",
                             input_length = max_length )
embedding

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding_2, built=False>

A rule of thumb in ML is to keep parameter vakues such that they are divisible by 8, the model works better.
For eg, batch of 32 size

In [ ]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nEmbedded text:\n")
# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Today was trauma on top of trauma on top of  trauma in Richmond so I know work is going to be crazy the next two days        

Embedded text:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01845245,  0.03416603,  0.03636669, ...,  0.02952019,
          0.00689237, -0.0269935 ],
        [ 0.04744576, -0.00487145, -0.01408532, ...,  0.02331522,
         -0.01454649,  0.00309715],
        [-0.02829355, -0.02086456,  0.03066445, ..., -0.03562853,
         -0.03683777, -0.00666194],
        ...,
        [-0.00154782, -0.00774702,  0.01865205, ..., -0.02090433,
         -0.04465425, -0.02777182],
        [-0.0197776 , -0.04827657,  0.03333825, ...,  0.01634354,
         -0.00954591, -0.03519575],
        [-0.03262522, -0.0361958 ,  0.04928818, ..., -0.00649082,
          0.00566211,  0.01994665]]], dtype=float32)>

In [ ]:
# Check out the embedding for a a single token
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01845245,  0.03416603,  0.03636669, -0.03340192, -0.004688  ,
         0.00313316,  0.00628572, -0.03215854,  0.00641593, -0.0130218 ,
        -0.03934449, -0.0215742 , -0.01057278, -0.03026411, -0.01549065,
        -0.0313793 ,  0.04379548,  0.02779596,  0.00865261,  0.02584286,
         0.01876714, -0.00422028,  0.02183333, -0.02437813,  0.0440745 ,
        -0.0242546 , -0.00481933, -0.0486388 ,  0.04226501, -0.0171304 ,
         0.00905817, -0.02521255,  0.04737115, -0.0115808 ,  0.02789129,
         0.00032517,  0.04451219,  0.01859505,  0.00788216,  0.00085925,
        -0.04161086,  0.04210789, -0.00043865, -0.00464649,  0.0315714 ,
        -0.00929434,  0.04654482, -0.01840841,  0.04666979, -0.01065204,
         0.00896396, -0.00728805, -0.04065462, -0.03062527,  0.0042609 ,
        -0.04701952,  0.0094988 ,  0.04487561, -0.00309036,  0.01658836,
        -0.02909141,  0.00542808,  0.03250109,  0.03796725, -0.03273602,
  

## Modelling a text dataset (running a series of experiments)

Now that we've got a way to turn our sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from Scikit Learn ML map.
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pre-trained Feature extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data


How will we approach these?
Use the standard steps in modelling with TensorFlow:
* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's importannt to create a baseline model so you've got a benchmark for the future models to built uponn...

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> **Note:** It's a common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if we can improve upon them.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Pipeline is like a guideline that do these steps in order
# Create tokenization and modeling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),  # Convert the words to numbers
    ("clf", MultinomialNB())       # Model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
train_sentences[0]

'@mogacola @zamtriossu i screamed after hitting tweet'

In [ ]:
train_labels[0]

0

In [ ]:
# Evlauate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline modle achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline modle achieves an accuracy of: 79.27%


In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

The metrics considered are:
* Accuracy
* Precision
* Recall
* F1-score

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall, and f1-score of a binary classification model.
    Ensures inputs are converted to NumPy arrays if necessary and handles device mismatches.
    """
    # Convert TensorFlow Tensors to NumPy arrays if necessary
    y_true = y_true.numpy() if hasattr(y_true, "numpy") else y_true
    y_pred = y_pred.numpy() if hasattr(y_pred, "numpy") else y_pred

    # Debugging information about device placement
    print(f"y_true device: {y_true.device if hasattr(y_true, 'device') else 'CPU'}")
    print(f"y_pred device: {y_pred.device if hasattr(y_pred, 'device') else 'CPU'}")

    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate other metrics
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

    # Prepare the results
    model_results = {
        "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1
    }

    return model_results


In [ ]:
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

y_true device: CPU
y_pred device: CPU


{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [ ]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [ ]:
from tensorflow.keras import layers

# Define the input layer
inputs = layers.Input(shape=(1,), dtype="string")

# Apply text vectorization
x = text_vectorizer(inputs)

# Apply the embedding layer
x = embedding(x)

# Add global average pooling to reduce the dimensionality
x = layers.GlobalAveragePooling1D()(x)

# Add the dense output layer with sigmoid activation
outputs = layers.Dense(1, activation="sigmoid")(x)

# Create the model
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [ ]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_5                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_2           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model_1_history = model_1.fit(x= train_sentences,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR,
                              experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20241223-022618
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.6433 - loss: 0.6498 - val_accuracy: 0.7454 - val_loss: 0.5373
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8016 - loss: 0.4683 - val_accuracy: 0.7861 - val_loss: 0.4706
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.8609 - loss: 0.3557 - val_accuracy: 0.7887 - val_loss: 0.4700
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8879 - loss: 0.2882 - val_accuracy: 0.7913 - val_loss: 0.4657
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9120 - loss: 0.2401 - val_accuracy: 0.7887 - val_loss: 0.4848


In [ ]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7802 - loss: 0.5184 


[0.48476797342300415, 0.7887139320373535]

In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


(762, 1)

In [ ]:
model_1_pred_probs[:10]

array([[0.29734462],
       [0.7785369 ],
       [0.99754983],
       [0.10358078],
       [0.10564689],
       [0.93746525],
       [0.93575144],
       [0.9923704 ],
       [0.96078706],
       [0.25065133]], dtype=float32)

In [ ]:
# Convert model prediction probabilites to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [ ]:
len(val_labels), len(model_1_preds)

(762, 762)

In [ ]:
print(f"val_labels device: {val_labels.device if hasattr(val_labels, 'device') else 'CPU'}")
print(f"model_1_preds device: {model_1_preds.device if hasattr(model_1_preds, 'device') else 'CPU'}")


val_labels device: CPU
model_1_preds device: /job:localhost/replica:0/task:0/device:CPU:0


In [ ]:
model_1_preds = tf.convert_to_tensor(model_1_preds).numpy() if tf.is_tensor(val_labels) else model_1_preds

In [ ]:
val_labels = val_labels.numpy() if hasattr(val_labels, "numpy") else val_labels
model_1_preds = model_1_preds.numpy() if hasattr(model_1_preds, "numpy") else model_1_preds


In [ ]:
print(f"val_labels device: {val_labels.device if hasattr(val_labels, 'device') else 'CPU'}")
print(f"model_1_preds device: {model_1_preds.device if hasattr(model_1_preds, 'device') else 'CPU'}")

val_labels device: CPU
model_1_preds device: CPU


In [ ]:
# Calculate our model_1 results
model_1_results =  calculate_results(y_true = val_labels,
                                     y_pred = model_1_preds)

model_1_results

y_true device: CPU
y_pred device: CPU


{'accuracy': 78.87139107611549,
 'precision': 0.7964015586347394,
 'recall': 0.7887139107611548,
 'f1': 0.7848945056280915}

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learnt embeddings

In [ ]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_5                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_2           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [ ]:
# Get the weight matrix of the embedding layer
# (these are the numerical representations of each token in out training data, which have been learnt for 5 epochs)
embed_weights = model_1.get_layer("embedding_2").get_weights()[0]
print(embed_weights.shape) # Should be the same as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)


In [ ]:
embed_weights

array([[ 0.01351533, -0.00321829,  0.04331478, ...,  0.04671112,
        -0.03705037, -0.04068276],
       [-0.02694126,  0.02409507,  0.03824852, ...,  0.04016338,
         0.02733284, -0.00598546],
       [ 0.01773582, -0.01710124,  0.04016459, ..., -0.04035642,
        -0.00688661,  0.00074076],
       ...,
       [-0.00602815,  0.02898533, -0.02057128, ..., -0.03882305,
        -0.00813171,  0.00779998],
       [ 0.05650644, -0.03992879,  0.06223193, ...,  0.03391041,
         0.04462701,  0.01340589],
       [ 0.02327464, -0.07011092,  0.10376947, ...,  0.03585747,
         0.0585922 , -0.07845342]], dtype=float32)

In [ ]:
# We use Projector by TensorFlow for the visualisations
# Create embedding files (we got this from TensorFlow's word embeddings documentation)
import io

out_v = io.open("vectors.tsv", "w", encoding = "utf-8")
out_m = io.open("metadata.tsv", "w", encoding = "utf-8")

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # skip 0, it's padding
  vec = embed_weights[index]
  out_v.write("\t".join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# # Download files from colab
# try:
#   from google.colab import files
#   files.download("vectors.tsv")
#   files.download("metadata.tsv")
# except Exception:
#   pass

## Recurrent Neural Networks (RNNs)

The premise of RNN is to use the representation of a previous input to aid the representation of a later input.

### Model 2: LSTM

LSTM = long short term memory (one of the most popular RNN cells)

A typical structure of an RNN looks something like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [ ]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # when stacking RNN cells, we need to set `return_sequences = True`
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation = "relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [ ]:
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_5                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
model_2.compile(loss= "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20241223-022703
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - accuracy: 0.8795 - loss: 0.3016 - val_accuracy: 0.7703 - val_loss: 0.6341
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.9454 - loss: 0.1509 - val_accuracy: 0.7861 - val_loss: 0.5736
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9588 - loss: 0.1167 - val_accuracy: 0.7913 - val_loss: 0.7079
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9652 - loss: 0.0965 - val_accuracy: 0.7730 - val_loss: 0.8904
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9645 - loss: 0.0871 - val_accuracy: 0.7782 - val_loss: 0.8596


In [ ]:
# Make predictions on the LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


array([[6.5559514e-02],
       [9.5357847e-01],
       [9.9975884e-01],
       [2.2369765e-01],
       [8.9785131e-04],
       [9.9939680e-01],
       [9.7344959e-01],
       [9.9984890e-01],
       [9.9975342e-01],
       [7.3530531e-01]], dtype=float32)

In [ ]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [ ]:
# Calculate model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)
model_2_results

y_true device: CPU
y_pred device: CPU


{'accuracy': 77.82152230971128,
 'precision': 0.7778849274912267,
 'recall': 0.7782152230971129,
 'f1': 0.7777458336465682}

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

Another popular and effective RNN component is GRU or Gated Recurrent Unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [ ]:
# Build an RNN using the GRU layer
from tensorflow.keras import layers
inputs = layers.Input(shape= (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# print(x.shape)
# x = layers.GRU(64, return_sequences = True)(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences = True)(x)
# print(x.shape)
# x = layers.GRU(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation = "relu")(x)
# print(x.shape)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation= "sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name= "model_3_GRU")

In [ ]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_5                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks  =[create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20241223-022749
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.8872 - loss: 0.2727 - val_accuracy: 0.7822 - val_loss: 0.7393
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 42ms/step - accuracy: 0.9710 - loss: 0.0775 - val_accuracy: 0.7769 - val_loss: 0.7450
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.9694 - loss: 0.0790 - val_accuracy: 0.7756 - val_loss: 0.9482
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - accuracy: 0.9726 - loss: 0.0680 - val_accuracy: 0.7730 - val_loss: 1.0838
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.9776 - loss: 0.0538 - val_accuracy: 0.7717 - val_loss: 1.1490


In [ ]:
# Make some predictions with model_3
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step


array([[1.3904281e-02],
       [7.8268033e-01],
       [9.9979717e-01],
       [6.7946643e-02],
       [1.0934823e-04],
       [9.9963087e-01],
       [6.9076467e-01],
       [9.9991882e-01],
       [9.9984068e-01],
       [7.9801267e-01]], dtype=float32)

In [ ]:
# Convert model 3 pred to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
# Calcualte model 3 results
model_3_results = calculate_results(y_true = val_labels,
                                    y_pred  =model_3_preds)
model_3_results

y_true device: CPU
y_pred device: CPU


{'accuracy': 77.16535433070865,
 'precision': 0.7726063176824562,
 'recall': 0.7716535433070866,
 'f1': 0.769953200185146}

### Model 4: Bidirectional RNN

In [ ]:
# Input layer
inputs = layers.Input(shape=(1,), dtype="string")

# Pass the input through the TextVectorization layer
x = text_vectorizer(inputs)

# Embedding layer
x = embedding(x)

# Bidirectional LSTM layer
x = layers.Bidirectional(layers.LSTM(64))(x)

# Output layer
outputs = layers.Dense(1, activation="sigmoid")(x)

# Define the model
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [ ]:
model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_5                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,378,945 (5.26 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_4.compile(loss= "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20241223-022855
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.9288 - loss: 0.2025 - val_accuracy: 0.7730 - val_loss: 0.9800
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - accuracy: 0.9759 - loss: 0.0536 - val_accuracy: 0.7730 - val_loss: 1.1963
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 58ms/step - accuracy: 0.9794 - loss: 0.0462 - val_accuracy: 0.7743 - val_loss: 1.2547
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9813 - loss: 0.0431 - val_accuracy: 0.7717 - val_loss: 1.0611
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - accuracy: 0.9796 - loss: 0.0460 - val_accuracy: 0.7664 - val_loss: 1.3878


In [ ]:
# Make predictions with out bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


array([[8.5692469e-04],
       [7.4988401e-01],
       [9.9992168e-01],
       [8.9808792e-02],
       [1.4617701e-05],
       [9.9815220e-01],
       [9.1589469e-01],
       [9.9996662e-01],
       [9.9993521e-01],
       [9.8860162e-01]], dtype=float32)

In [ ]:
# Convert pred probs to label
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
# Calculate the results of our bidirectional model
model_4_results = calculate_results(y_true  =val_labels,
                                    y_pred = model_4_preds)
model_4_results

y_true device: CPU
y_pred device: CPU


{'accuracy': 76.64041994750657,
 'precision': 0.7674517658976838,
 'recall': 0.7664041994750657,
 'f1': 0.7645419362124183}

## Cnvolutional Neural Networks for Text (and other types of sequences)

As opposed to images, our text data is 1D. Hence, we will be using Conv1D instead of Conv2D layer.

A typical structure of a Conv1D model for sequences:
```
Inputs (texts) -> Tokenize -> Embedding -> Layer (s) (typically Conv1D + pooling) -> Outputs (class probabilities)

### Model 5: Conv1D

To understand more about the parameters of Conv layers, refer cnnexplainer. Although it is for 2D, it can be used for understanding the concepts

To understand the difference between padding types: https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t

In [ ]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters = 64,
                        kernel_size =5,
                        activation= "relu",
                        padding = "valid") # if padding = valid, then output is smaller. if padding = same, then output is the same size as the input
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 64]), TensorShape([1, 64]))

In [ ]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04307893, -0.00198375,  0.00692423, ...,  0.02082079,
          0.10048071,  0.03265424],
        [ 0.0076906 , -0.01467775,  0.00623852, ..., -0.01771214,
         -0.03729355, -0.02333973],
        [ 0.07000521, -0.03060058,  0.06818248, ...,  0.03609796,
         -0.05851227,  0.0024743 ],
        ...,
        [ 0.02398193,  0.00864149,  0.02390374, ..., -0.00744138,
          0.00343362, -0.00233179],
        [ 0.02398193,  0.00864149,  0.02390374, ..., -0.00744138,
          0.00343362, -0.00233179],
        [ 0.02398193,  0.00864149,  0.02390374, ..., -0.00744138,
          0.00343362, -0.00233179]]], dtype=float32)>

In [ ]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 64), dtype=float32, numpy=
array([[[0.        , 0.01955422, 0.13216026, 0.        , 0.        ,
         0.        , 0.03640549, 0.        , 0.        , 0.        ,
         0.02704967, 0.        , 0.01681415, 0.        , 0.        ,
         0.00860578, 0.01916246, 0.        , 0.        , 0.        ,
         0.03420249, 0.03937607, 0.        , 0.        , 0.        ,
         0.        , 0.07744347, 0.00985504, 0.        , 0.        ,
         0.0172083 , 0.03906582, 0.        , 0.        , 0.03814223,
         0.        , 0.        , 0.        , 0.06019508, 0.08169326,
         0.00791387, 0.        , 0.        , 0.        , 0.00078626,
         0.06983025, 0.        , 0.        , 0.04766768, 0.        ,
         0.        , 0.0092722 , 0.        , 0.02470201, 0.02597667,
         0.        , 0.00201913, 0.        , 0.0129708 , 0.03259044,
         0.        , 0.        , 0.        , 0.02295062],
        [0.        , 0.        , 0.08230177, 0.        , 0.00

In [ ]:
max_pool_output

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[0.        , 0.05131678, 0.13216026, 0.00902945, 0.00105888,
        0.02277845, 0.05448944, 0.        , 0.07103466, 0.02263813,
        0.02704967, 0.02661701, 0.05306081, 0.06697194, 0.        ,
        0.00860578, 0.02195159, 0.03416384, 0.0256108 , 0.00950417,
        0.03420249, 0.08123844, 0.03414986, 0.01178903, 0.02268756,
        0.0101469 , 0.07744347, 0.08150522, 0.0317952 , 0.05308765,
        0.03574815, 0.03906582, 0.00440775, 0.07114444, 0.11563541,
        0.01598096, 0.0462285 , 0.06828666, 0.06019508, 0.08169326,
        0.01595135, 0.        , 0.01665472, 0.00055953, 0.02867029,
        0.06983025, 0.        , 0.        , 0.04766768, 0.03795229,
        0.00519545, 0.0092722 , 0.05706931, 0.03267264, 0.04840123,
        0.03460258, 0.02914374, 0.02137263, 0.02533902, 0.03259044,
        0.01612826, 0.        , 0.03991328, 0.08068142]], dtype=float32)>

In [ ]:
# Create 1-dimensional convolutional layer to model sequences

# Input layer
inputs = layers.Input(shape=(1,), dtype="string")

# Pass the input through the TextVectorization layer
x = text_vectorizer(inputs)

# Embedding layer
x = embedding(x)

# Bidirectional LSTM layer
x = layers.Conv1D(filters = 64,
                  kernel_size =5,
                  strides = 1,
                  activation= "relu",
                  padding = "valid")(x)

# MaxPoll layer
x = layers.GlobalMaxPool1D()(x)

# Output layer
outputs = layers.Dense(1, activation="sigmoid")(x)

# Define the model
model_5 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [ ]:
# Compile the model
model_5.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model_5.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_5                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_6 (Conv1D)                    │ (None, 11, 64)              │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_4               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,321,089 (5.04 MB)

 Trainable params: 1,321,089 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks =[create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20241223-023047
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.9402 - loss: 0.1803 - val_accuracy: 0.7717 - val_loss: 0.9013
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.9767 - loss: 0.0657 - val_accuracy: 0.7638 - val_loss: 1.0311
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.9766 - loss: 0.0587 - val_accuracy: 0.7664 - val_loss: 1.1336
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.9804 - loss: 0.0517 - val_accuracy: 0.7559 - val_loss: 1.1490
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.9772 - loss: 0.0488 - val_accuracy: 0.7638 - val_loss: 1.2075


In [ ]:
# Make some predictions with Conv1D model
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


array([[3.2874070e-02],
       [9.1849226e-01],
       [9.9990332e-01],
       [4.0065341e-02],
       [9.1980006e-08],
       [9.9051571e-01],
       [9.9022883e-01],
       [9.9996692e-01],
       [9.9999917e-01],
       [7.5049937e-01]], dtype=float32)

In [ ]:
# Convert the probabilities to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [ ]:
# Calculate the results of Conv1D model
model_5_results = calculate_results(y_true = val_labels,
                                    y_pred= model_5_preds)
model_5_results

y_true device: CPU
y_pred device: CPU


{'accuracy': 76.37795275590551,
 'precision': 0.7647678424871814,
 'recall': 0.7637795275590551,
 'f1': 0.761896339990086}

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}